In [87]:
import pandas as pd
import os
import numpy as np
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder

In [88]:
df = pd.read_excel('Test.xlsx')
df

,Tarih (Date),Gün (String),Para Birimi,Reklam Harcaması (TL),Reklamdan Kazanç (TL)
0,2019-01-01,Tuesday,TRY,215520,1918128.00
1,2019-01-02,Wednesday,TRY,143270,2616110.20
2,2019-01-03,Thursday,TRY,247056,1899860.64
3,2019-01-04,Friday,TRY,18752,346912.00
4,2019-01-05,Saturday,TRY,210615,1832350.50
...,...,...,...,...,...
1967,2024-05-21,Tuesday,TRY,127551,2269132.29
1968,2024-05-22,Wednesday,TRY,150380,2669245.00
1969,2024-05-23,Thursday,TRY,255274,3928666.86
1970,2024-05-24,Friday,TRY,237909,3295039.65


In [3]:
# @title cost vs revenue

#df.plot(kind='scatter', x='cost', y='revenue', s=32, alpha=.8)
#plt.gca().spines[['top', 'right',]].set_visible(False)

In [4]:
# @title revenue

#df['revenue'].plot(kind='hist', bins=20, title='revenue')
#plt.gca().spines[['top', 'right',]].set_visible(False)

In [5]:

#df['cost'].plot(kind='hist', bins=20, title='cost')
#plt.gca().spines[['top', 'right',]].set_visible(False)

In [90]:
df.dtypes

Tarih (Date)             datetime64[ns]
Gün (String)                     object
Para Birimi                      object
Reklam Harcaması (TL)             int64
Reklamdan Kazanç (TL)           float64
dtype: object

In [91]:
df.columns = ['date', 'day', 'curr', 'cost', 'revenue']

In [92]:
df.head()

,date,day,curr,cost,revenue
0,2019-01-01,Tuesday,TRY,215520,1918128.00
1,2019-01-02,Wednesday,TRY,143270,2616110.20
2,2019-01-03,Thursday,TRY,247056,1899860.64
3,2019-01-04,Friday,TRY,18752,346912.00
4,2019-01-05,Saturday,TRY,210615,1832350.50


In [93]:
df = df.drop(['curr', 'date'], axis=1)

In [94]:
df

,day,cost,revenue
0,Tuesday,215520,1918128.00
1,Wednesday,143270,2616110.20
2,Thursday,247056,1899860.64
3,Friday,18752,346912.00
4,Saturday,210615,1832350.50
...,...,...,...
1967,Tuesday,127551,2269132.29
1968,Wednesday,150380,2669245.00
1969,Thursday,255274,3928666.86
1970,Friday,237909,3295039.65


In [101]:
label_encoder = LabelEncoder()
df['day'] = label_encoder.fit_transform(df['day'])

In [102]:
df.isnull().sum()

day        0
cost       0
revenue    0
dtype: int64

In [103]:
X = df.drop(['revenue'], axis=1)
X

,day,cost
0,5,215520
1,6,143270
2,4,247056
3,0,18752
4,2,210615
...,...,...
1967,5,127551
1968,6,150380
1969,4,255274
1970,0,237909


In [104]:
y = df['revenue']
y

0       1918128.00
1       2616110.20
2       1899860.64
3        346912.00
4       1832350.50
           ...    
1967    2269132.29
1968    2669245.00
1969    3928666.86
1970    3295039.65
1971     362332.16
Name: revenue, Length: 1972, dtype: float64

In [105]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.3,random_state=0)

In [106]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

In [107]:
lr.fit(X_train, y_train)


LinearRegression()

In [121]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

y_pred = lr.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

performance_metrics = pd.DataFrame({'Metric': ['MSE', 'RMSE', 'MAE', 'R2'],
                                    'Value': [mse, rmse, mae, r2]})
print(performance_metrics)


  Metric         Value
0    MSE  3.128165e+11
1   RMSE  5.593000e+05
2    MAE  4.420345e+05
3     R2  7.095106e-01


In [122]:
performance_metrics['Value'] = performance_metrics['Value'].map('{:,.2f}'.format)


In [123]:
print(performance_metrics)


  Metric               Value
0    MSE  312,816,488,435.20
1   RMSE          559,300.00
2    MAE          442,034.47
3     R2                0.71


In [126]:
# Performans metriklerini bir sözlükte toplayın
performance_metrics = {
    'MSE': mse,
    'RMSE': rmse,
    'R2': r2,
    'MAE': mae
}

In [127]:
# Performans metriklerini JSON dosyasına kaydedin
import json
with open('performance_metrics.json', 'w') as f:
    json.dump(performance_metrics, f)

In [111]:
sample_data = pd.DataFrame({'day': [2], 'cost': [4865]})

sample_prediction = lr.predict(sample_data)

print('Sample Prediction:', sample_prediction)


Sample Prediction: [77376.3813901]


In [112]:
"{:,.2f} TL".format(sample_prediction[0])

'77,376.38 TL'

In [113]:
pickle.dump(lr, open("model.pkl" , "wb"))